Vous trouverez dans ce notebook la totalité des scripts ayant servi au prétraitement des données

Voici la ligne de commande MFA qui a servi à l'alignement par phonème

Ce script a permis d'harmoniser le nom des tier qui n'était pas les mêmes. En effet les données ont été sujette à beaucoup de révisions à la main ce qui peut causer des disfluences lorsque les textgrids sont sauvegardé.

In [ ]:
import textgrid
import os
def extraire_tier(textgrid_obj, tier_name):

    for tier in textgrid_obj.tiers:
        if tier.name == tier_name:
            return tier
    return None

def modifier_tier(file_path, tier_name):

    textgrid_obj = textgrid.TextGrid.fromFile(file_path)

    tier = extraire_tier(textgrid_obj, tier_name)

    if tier is not None:
        for interval in tier.intervals:
            if interval.mark.startswith('aj'): #permet d'harmoniser le nom des annotations
                interval.mark = 'ajajaj'
            elif interval.mark.startswith('uj'):
                interval.mark = 'ujujuj'
            elif interval.mark.startswith('wi'):
                interval.mark = 'wiwiwi'

        textgrid_obj.write(file_path)
        print("ok")
    else:
        print("pasok")

dossier_textgrid = "CTRL"

for fichier in os.listdir(dossier_textgrid):
    if fichier.endswith(".TextGrid"):
        chemin_fichier = os.path.join(dossier_textgrid, fichier)
        modifier_tier(chemin_fichier, "words") #permet de choisir le tier


Une fois les Textgrids nettoyé nous pouvons extraire les données cepstraux. Le script a été donné par Mr Gendrot puis revisité pour pouvoir le lancer sur une arborescence de fichier. Nous avons une version praat et python pour la lecture de cette modulation. Neanmoins aucun des deux scripts fonctionnent mieux que l'autre. Nous choisissons notre préféré :D

In [ ]:
#from get_MFCCS_change import get_MFCCS_change



# -*- coding: utf-8 -*-
"""
Created on Thu Jun  9 08:45:25 2022

@author: Leonardo
"""

import os
import glob
import numpy as np
from librosa.feature import mfcc
from librosa.core import load as audioLoad
from scipy import signal



def get_MFCCS_change(filePath=None, channelN=0, sigSr=10000, tStep=0.005, winLen=0.025, n_mfcc=13, n_fft=512, removeFirst=1, filtCutoff=12, filtOrd=6):
    """ computes the amount of change in the MFCCs over time
    
    INPUT:
        
    filePath(default="./signals/audio/loc01_pert1_1_2_3_6_01.wav"
    channelN (default(default=0): selet the channel number for multichannel audio file
    sigSr (default=10000): frequency at which resampling the audio for the analysis
    tStep (default=0.005): analysis time step in ms
    winLen (default=0.025): analysis window length in ms
    n_mfcc (default=13): number of MFCCs to compute (the first one may then be removed via reoveFirst)
    n_fft (default=512): number of points for the FFT
    removeFirst (default=1): if one the first cepstral corefficient is dicarded
    filtCutoff (default=12): bandpass fitler freq.in Hz
    filtOrd (default=6): bandpass filter order
    
    OUTPUT:
    
    totChange: Amount of change over time
    T: time stamps for each value   
    """
    
    myAudio, _ = audioLoad(filePath,sr=sigSr, mono=False)
    
    if len(np.shape(myAudio))>1:
        y=myAudio[channelN,:]
    else:
        y=myAudio
    
    win_length=np.rint(winLen*sigSr).astype(int)
    hop_length=np.rint(tStep*sigSr).astype(int)
    
    
    myMfccs=mfcc( y=y, sr=sigSr, n_mfcc=n_mfcc, win_length=win_length, hop_length=hop_length,n_fft=n_fft)
    
    T=np.round(np.multiply(np.arange(1,np.shape(myMfccs)[1]+1),tStep)+winLen/2,4)
    
    if removeFirst:
       myMfccs=myMfccs[1:,:]
       
    cutOffNorm = filtCutoff / ((1/tStep) / 2)
    
    b1,a1 =signal.butter(filtOrd,cutOffNorm, 'lowpass')
    
    filtMffcs = signal.filtfilt(b1,a1,myMfccs)
    
    myAbsDiff=np.sqrt(np.gradient(filtMffcs,axis=1)**2)
    
    totChange=np.sum(myAbsDiff,0)
    
    totChange=signal.filtfilt(b1,a1,totChange)
    
    return totChange, T
    #plt.plot(totChange)
#Modification effectué par nous
dossier_principal = "pourNatacha"

def parcourir_arborescence_dossier(dossier):
    """
    Parcourt l'arborescence du dossier spécifié et applique la fonction get_MFCCS_change() sur les fichiers .wav.
    Enregistre les résultats dans des fichiers .txt avec le même nom que les fichiers .wav.
    """
    for dossier_actuel, sous_dossiers, fichiers in os.walk(dossier):
        for fichier in fichiers:
            if fichier.endswith(".wav"):
                chemin_fichier = os.path.join(dossier_actuel, fichier)
                print("Chemin du fichier .wav :", chemin_fichier)
                spectralChange, timeStamps = get_MFCCS_change(chemin_fichier)
                nom_fichier_resultat = os.path.splitext(fichier)[0] + ".txt"
                chemin_fichier_resultat = os.path.join(dossier_actuel, nom_fichier_resultat)
                np.savetxt(chemin_fichier_resultat, spectralChange)

parcourir_arborescence_dossier(dossier_principal)

#print(0)
#print(np)

Le script permettant de visualiser la modulation qui est sous forme textuelle. Celui-ci nous permettra par la suite de pouvoir interpreter l'intêret de la modulation cepstrale pour un modèle fonctionnant avec des images

In [ ]:
import numpy as np
from scipy.io import wavfile
import os

def modulation_mfcc(fichier_mfcc):
    with open(fichier_mfcc, 'r') as f:
        valeurs_mfcc = [float(line.strip()) for line in f]

    valeurs = np.array(valeurs_mfcc)

    # Paramètres audio
    fs = 200  # Fréquence d'échantillonnage en Hz

    # Remove .txt extension from filename
    nom_fichier_wav = os.path.splitext(fichier_mfcc)[0] + "_modulation.Sound"
    wavfile.write(nom_fichier_wav, fs, valeurs.astype(np.float64))
    return nom_fichier_wav





def parcourir_arborescence_dossier(dossier):
    """
    Parcourt l'arborescence du dossier spécifié et applique la fonction modulation_mfcc() sur les fichiers .txt.
    Enregistre les résultats dans des fichiers .Sound avec le nom de fichiers fichiertxt_modulation.Sound.
    """
    for dossier_actuel, sous_dossiers, fichiers in os.walk(dossier):
        for fichier in fichiers:
            if fichier.endswith(".txt"):
                chemin_fichier = os.path.join(dossier_actuel, fichier)        
                modulation = modulation_mfcc(chemin_fichier)
                print("modulation cepstrale :", modulation)

dossier_principal = "pourNatacha"
parcourir_arborescence_dossier(dossier_principal)


Voici la version praat

In [ ]:
clearinfo
wd$="pourNatacha/"
Create Strings as folder list: "list", wd$
select Strings list
nFolder = Get number of strings
for iFolder from 1 to nFolder
	select Strings list
	fName$ = Get string: iFolder
	wd2$ = wd$+fName$+"/"
	Create Strings as file list: "listmeasure", wd2$+"*.txt"
	select Strings listmeasure
	nFiles = Get number of strings
		for iFile from 1 to nFiles
   			select Strings listmeasure
    			fName$ = Get string: iFile
			appendInfoLine: wd2$ + fName$
			Read Table from tab-separated file: wd2$ + fName$ 
    			Down to Matrix
    			Transpose
    			To Sound
    			Override sampling frequency: 200
    			Shift times by: -0.5
			
			Save as binary file:  wd2$+fName$ -".txt"+"modulation.Sound"

    		
			endfor
    		
	endfor


Maintenant les scripts qui ont extrait la modulation cepstrale pour la transformer en bande montrant le comportement de sa courbe initiale, c'est à dire son intensité et sa répartition temporelle.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from praatio import tgio

def get_mfcc_values(mfcc_filename, textgrid_filename):
    with open(mfcc_filename, 'r') as f:
        valeurs_mfcc = [float(line.strip()) for line in f]

    tg = tgio.openTextgrid(textgrid_filename)
    start_time, end_time = tg.tierDict["'Segmentation'"].entryList[0][0], tg.tierDict["'Segmentation'"].entryList[-1][1]
    start_index = int(start_time * 200) 
    end_index = int(end_time * 200)

    portion_mfcc = valeurs_mfcc[start_index:end_index]
    return portion_mfcc

def normalize_mfcc_values(portion_mfcc, global_min, global_max):
    normalized_values = [(x - global_min) / (global_max - global_min) for x in portion_mfcc]
    return normalized_values

sca_folder = "SCA/glides"
ctrl_folder = "CTRL/glides"

global_min = float('inf')
global_max = float('-inf')

for folder in [sca_folder, ctrl_folder]:
    for root, dirs, files in os.walk(folder):
        for file_name in files:
            if file_name.endswith(".txt"):
                mfcc_file_path = os.path.join(root, file_name)
                
                textgrid_file_path = os.path.join(root, file_name.replace(".txt", ".TextGrid"))
                
                portion_mfcc = get_mfcc_values(mfcc_file_path, textgrid_file_path)
                local_min = min(portion_mfcc)
                local_max = max(portion_mfcc)
                
                if local_min < global_min:
                    global_min = local_min
                    file_min = file_name
                
                if local_max > global_max:
                    global_max = local_max
                    file_max = file_name
                
                print(f"Fichier: {file_name}, Min: {local_min}, Max: {local_max}")

print(f"Valeur Min Globale: {global_min} (Fichier: {file_min})")
print(f"Valeur Max Globale: {global_max} (Fichier: {file_max})")

for folder in [sca_folder, ctrl_folder]:
    for root, dirs, files in os.walk(folder):
        for file_name in files:
            if file_name.endswith(".txt"):
                mfcc_file_path = os.path.join(root, file_name)
                textgrid_file_path = os.path.join(root, file_name.replace(".txt", ".TextGrid"))
                
                portion_mfcc = get_mfcc_values(mfcc_file_path, textgrid_file_path)
                normalized_values = normalize_mfcc_values(portion_mfcc, global_min, global_max)
                
                fig, ax = plt.subplots(figsize=(300, 1))  
                ax.imshow([normalized_values], cmap='gray_r', aspect='auto')
                ax.set_axis_off()

                output_folder = "malade_glides" if "SCA" in mfcc_file_path else "sain_glides"

                output_path = os.path.join(output_folder, f"{file_name}_output_image.png")
                plt.savefig(output_path, bbox_inches='tight', pad_inches=0)

                plt.close()


Celui pour les spectrogrammes : 

In [ ]:
import os
import parselmouth
import numpy as np
import matplotlib.pyplot as plt
from praatio import tgio
import glob


#source documentation parselmouth
def spectrograme(spectrogram, filename, image_width_inches, image_height_inches, resolution_dpi=100, dynamic_range=160, zero_padding_width_ratio=0, value_for_zero_padding=None):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
#ajout pour eviter le padding
    if 0 < zero_padding_width_ratio < 1:
        if value_for_zero_padding is None:
            value_for_zero_padding = np.min(sg_db)
        
        n_rows, n_cols = sg_db.shape
        n_added_cols = int(np.round(n_cols * zero_padding_width_ratio / (1 - zero_padding_width_ratio)))
        added_zeros = np.full((n_rows, n_added_cols), value_for_zero_padding)
        zero_padded_sg_db = np.flipud(np.concatenate((sg_db, added_zeros), axis=1))
    else:
        zero_padded_sg_db = np.flipud(sg_db)

    fig, ax = plt.subplots(frameon=False)
    fig.set_size_inches(image_width_inches, image_height_inches)
    ax.set_axis_off()
    ax.imshow(zero_padded_sg_db, aspect="auto", cmap="gray_r", vmin=sg_db.max() - dynamic_range, vmax=None)
    plt.savefig(filename, bbox_inches='tight', pad_inches=0, dpi=resolution_dpi)
    plt.close()
#pour prendre tout le spectrogramme sur la durée de l'annotation
def extract_total_spectrograms(audio_file, textgrid_file, output_folder, dynamic_range):
    sound = parselmouth.Sound(audio_file)
    tg = tgio.openTextgrid(textgrid_file)
    phon_tier = tg.tierDict['words']

    for idx, interval in enumerate(phon_tier.entryList):
        label = interval.label
        if label.lower() in ['ajajaj', 'ujujuj', 'wiwiwi','bababa',"badegobadegobadgeo","dedede","gogogo"]:
            start_time = interval.start
            end_time = interval.end

            extract_s = sound.extract_part(from_time=start_time, to_time=end_time)

            max_frequency_spectrogram_Hz = 8000
            spectrogram = extract_s.to_spectrogram(maximum_frequency=max_frequency_spectrogram_Hz)

            audio_file_name = os.path.splitext(os.path.basename(audio_file))[0]
            output_filename = f"{output_folder}/spectrogram_{audio_file_name}_{label}_{idx}.png"

            spectrograme(
                spectrogram, filename=output_filename,
                image_width_inches=8, image_height_inches=6, dynamic_range=110)
                #l'image sera à nouveau remodeler avec le CNN.
def process_patient_folder(patient_folder, output_folder, dynamic_range):
    for root, dirs, files in os.walk(patient_folder):
        for file in files:
            if file.lower().endswith(".wav"):
                audio_file = os.path.join(root, file)
                textgrid_file = os.path.join(root, file.replace(".wav", ".TextGrid"))

                if os.path.exists(textgrid_file):
                    extract_total_spectrograms(audio_file, textgrid_file, output_folder, dynamic_range)

def main():
    input_folder = 'glides/'
    output_folder = 'test'
    dynamic_range = 120  

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for patient_folder in glob.glob(os.path.join(input_folder, '*/')):
        process_patient_folder(patient_folder, output_folder, dynamic_range)

if __name__ == "__main__":
    main()
